# Sentiment Analysis con PCA
## Clasificador en C++
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: no se puede crear el directorio «build»: El archivo ya existe
-- The C compiler identification is GNU 8.3.0
-- The CXX compiler identification is GNU 8.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /usr/bin/python3 (found version "3.6.8") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/guidotripodi/Documentos/Facu/MN/mntp2/mnv-tp2
-- Configuring done

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home/guidotripodi/Documentos/Facu/MN/mntp2/mnv-tp2/notebooks
Python 2.7.16


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: No se puede efectuar open: No existe el archivo o el directorio
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,The unthinkable has happened. Having first wit...,neg,7721_4.txt
freq,6275,2,6322,2


In [5]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [8]:
import sentiment

# Corremos PCA
alpha = 50
pca = sentiment.PCA(alpha)

pca.fit(X_train.toarray())
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import sentiment
from sklearn.metrics import accuracy_score

vectorizer = CountVectorizer(max_df=0.8, min_df=0.05, max_features=5000)

vectorizer.fit(text_train)

X_train4, y_train4 = vectorizer.transform(text_train), (label_train == 'pos').values
X_test4, y_test4 = vectorizer.transform(text_test), (label_test == 'pos').values

x_values = [5,10,20,50,100,500,1000,2000,3000,4000,5000,6500]

scores = []
k_values = [1,5,10,15,20,25,50] + list(range(100,400,50))
alpha_values = [50,100,150]
for x in x_values:
    x_train4Aux = X_train4
    y_train4Aux = y_train4    
    x_train4Aux = x_train4Aux[0:x]
    y_train4Aux = y_train4Aux[0:x]
    for alpha in alpha_values:
        for k in k_values:
            if k >= x:
                break
            x_train4Aux = X_train4
            y_train4Aux = y_train4
            x_train4Aux = x_train4Aux[0:x]
            y_train4Aux = y_train4Aux[0:x]
            X_test4aux = X_test4
            y_test4aux = y_test4

            pca = sentiment.PCA(alpha)
            pca.fit(x_train4Aux.toarray())

            x_train4Aux = pca.transform(x_train4Aux)
            X_test4aux = pca.transform(X_test4aux)

            clf = sentiment.KNNClassifier(k)
            clf.fit(x_train4Aux, y_train4Aux)

            y_pred4 = clf.predict(X_test4aux)
            acc = accuracy_score(y_test4aux, y_pred4)
        
            print("k: {}   acc: {}  Alpha: {}".format(k, acc, alpha))
            scores.append(acc)

print(scores)

k: 1   acc: 0.4946613545816733  Alpha: 50
k: 1   acc: 0.4887649402390438  Alpha: 100
k: 1   acc: 0.49163346613545816  Alpha: 150
k: 1   acc: 0.4964143426294821  Alpha: 50
k: 5   acc: 0.5083665338645418  Alpha: 50
k: 1   acc: 0.4946613545816733  Alpha: 100
k: 5   acc: 0.49545816733067727  Alpha: 100
k: 1   acc: 0.5214342629482072  Alpha: 150
k: 5   acc: 0.4949800796812749  Alpha: 150
k: 1   acc: 0.5133067729083666  Alpha: 50
k: 5   acc: 0.5136254980079681  Alpha: 50
k: 10   acc: 0.49306772908366536  Alpha: 50
k: 15   acc: 0.49354581673306774  Alpha: 50
k: 1   acc: 0.514581673306773  Alpha: 100
k: 5   acc: 0.5136254980079681  Alpha: 100
k: 10   acc: 0.49338645418326693  Alpha: 100
k: 15   acc: 0.49354581673306774  Alpha: 100
k: 1   acc: 0.5163346613545817  Alpha: 150
k: 5   acc: 0.5131474103585657  Alpha: 150
k: 10   acc: 0.4932270916334661  Alpha: 150
k: 15   acc: 0.49354581673306774  Alpha: 150
k: 1   acc: 0.52  Alpha: 50
k: 5   acc: 0.5400796812749004  Alpha: 50
k: 10   acc: 0.5281274

k: 350   acc: 0.6420717131474104  Alpha: 50
k: 1   acc: 0.569402390438247  Alpha: 100
k: 5   acc: 0.5966533864541833  Alpha: 100
k: 10   acc: 0.611792828685259  Alpha: 100
k: 15   acc: 0.6239043824701195  Alpha: 100
k: 20   acc: 0.6286852589641434  Alpha: 100
k: 25   acc: 0.6291633466135458  Alpha: 100
k: 50   acc: 0.6438247011952192  Alpha: 100
k: 100   acc: 0.648605577689243  Alpha: 100
k: 150   acc: 0.651792828685259  Alpha: 100
k: 200   acc: 0.6484462151394422  Alpha: 100
k: 250   acc: 0.6463745019920318  Alpha: 100
k: 300   acc: 0.6462151394422311  Alpha: 100
k: 350   acc: 0.6447808764940239  Alpha: 100
k: 1   acc: 0.5679681274900399  Alpha: 150
k: 5   acc: 0.5899601593625498  Alpha: 150
k: 10   acc: 0.6094023904382471  Alpha: 150
k: 15   acc: 0.628207171314741  Alpha: 150
k: 20   acc: 0.6387250996015936  Alpha: 150
k: 25   acc: 0.6411155378486055  Alpha: 150
k: 50   acc: 0.6505179282868526  Alpha: 150
k: 100   acc: 0.6516334661354581  Alpha: 150
k: 150   acc: 0.6438247011952192  

In [10]:
# Corremos KNN
clf = sentiment.KNNClassifier(5)

clf.fit(X_train, y_train)


In [11]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.6058964143426295
CPU times: user 5.33 s, sys: 12 ms, total: 5.34 s
Wall time: 5.32 s
